## Description

This notebook details the analytical and monte carlo methods of computing the average length of travel of a particle in a silicone detector.

## Dependencies

In [35]:
import numpy as np
import matplotlib.pyplot as plt

from typing import Tuple

## Generating points and direction vectors for the monte carlo method

In [38]:
def uniform_points_on_circle(num_points, radius, center = (0, 0)):
    """Generates a list of coordinates of points on a circle"""
    theta = np.random.uniform(0, 2 * np.pi, num_points)
    r = radius * np.sqrt(np.random.uniform(0, 1, num_points))
    
    x_0, y_0 = center

    x = r * np.cos(theta) + x_0
    y = r * np.sin(theta) + y_0
    z = np.zeros(num_points)    
    return list(zip(x, y, z))

In [39]:
def uniform_normal_vector():
    """Generates a uniform normal vector pointing down"""
    
    # Generate three independent random numbers from a normal distribution (Gaussian distribution)
    v = np.random.normal(0, 1, 3)
    v /= np.linalg.norm(v)
    v[2] = -v[2] if v[2] > 0 else v[2]
    
    return v

## Computing intersections

In [94]:
def cylinder_intersections(line_origin, line_direction, cylinder_origin, cylinder_radius, cylinder_depth):
    x_0, y_0, z_0 = line_origin
    d_x, d_y, d_z = line_direction
    x_c, y_c, z_c = cylinder_origin
    r = cylinder_radius

    a = d_x * d_x + d_y * d_y
    b = 2 * ((x_0 - x_c) * d_x + (y_0 - y_c) * d_y)
    c = (x_0 - x_c)**2 + (y_0 - y_c)**2 - r * r

    delta = b * b - 4 * a * c
    
    # One intersection
    if delta < 0:
        ts = []
    elif delta <= 10e-8:
        ts = [(-b + delta ) / (2 * a)]
    else:
        ts = [
            ( -b + delta ) / (2 * a),
            ( -b - delta ) / (2 * a),
        ]

    
    intersections = [(x_0 + t * d_x, y_0 + t * d_y, z_0 + t * d_z) for t in ts]
    intersections = [inter for inter in intersections if inter[2] > z_c - cylinder_depth and inter[2] < z_c]

    return intersections
    

In [95]:
def circle_intersections(line_origin, line_direction, circle_origin, circle_radius):
    x_0, y_0, z_0 = line_origin
    d_x, d_y, d_z = line_direction
    x_c, y_c, z_c = circle_origin
    r = circle_radius

    t = (z_c - z_0) / d_z
    x_p = x_0 + t * d_x
    y_p = y_0 + t * d_y
    z_p = z_c

    if (x_p - x_c)**2 + (y_p - y_c)**2 <= r:
        return [(x_p, y_p, z_p)]
    else:
        return []

## Monte Carlo method for computing average particle travel distance

In [96]:
surface_samples = 1_000
direction_samples = 1_000


# "Detector stack contains eight Si diodes, first one of the 3 mm, others of 6 mm diameter" ~ Patryk Socha Phd. Page 29
diode_radius = 6  # [mm]
diode_depth = 0.3 # [mm]
diode_top_origin = (0, 0, 0) # [mm]
diode_bottom_origin = (0, 0, -diode_depth) # [mm]

max_distance = np.sqrt(diode_depth**2 + 4 * diode_radius**2)

origins = uniform_points_on_circle(surface_samples, diode_radius)
distance_sum = 0.0

for origin in origins:
    direction_vectors = [uniform_normal_vector() for _ in range(direction_samples)]
    for direction in direction_vectors:
        wall_intersections = cylinder_intersections(origin, direction, diode_top_origin, diode_radius, diode_depth)
        bottom_intersections =  circle_intersections(origin, direction, diode_bottom_origin, diode_radius)
        intersections = wall_intersections + bottom_intersections
        
        if len(intersections) > 0:
            intersection_distance_sum = 0.0
            intersections_count = 0
            o_x, o_y, o_z = origin
            for i in intersections:
                i_x, i_y, i_z = i
                distance = (i_x-o_x)**2 + (i_y-o_y)**2 + (i_z-o_z)**2
                if distance < max_distance:
                    intersection_distance_sum += distance
            distance_sum += intersection_distance_sum / len(intersections)

average_distance = distance_sum / (surface_samples * direction_samples)
print(f"Average distance is: {average_distance}mm with a {average_distance / diode_depth} ratio")

        

Average distance is: 0.15951619717505566mm with a 0.5317206572501856 ratio
